In [1]:
import os 
import pandas as pd
import re
import numpy as np

In [ ]:
name_pattern = '(?s:.*)(ssl|init|lfs|delay)'
probability_pattern  = '\d\.\d{1,2}'

In [2]:
def get_type(x):
    return re.search(name_pattern,x).group(1)

def get_probability(x):
    if re.search(probability_pattern,x):
        return re.search(probability_pattern,x).group(0)

In [106]:

def prepare_logs(logs_file,info_from_hyperparameters = []):

    # read data
    res = pd.read_csv(os.path.join('logs',logs_file),header= None)

    # renaming columns
    res = res.rename(columns={0:'stream',1:'hyperparameters',2:'B',3:'frequency'})
    res['hyperparameters'] = res['hyperparameters'].apply(lambda x: dict([i.split(";") for i in re.findall( '\((.*?)\)',x)]))
    B = res['B'].iloc[0]
    cl = res.iloc[:,B+6:].columns
    res = res.rename(columns = dict(zip(res.columns[4:B+6],[f"B_{i}" for i in range(1,B+3)])))
    res = res.rename(columns = dict(zip(cl,[f"m_{i}" for i in range(len(cl))])))

    # adding new columns
    res['date'] = res['stream'].apply(lambda x: x.split(' - ')[0])
    res['stream'] = res['stream'].apply(lambda x: x.split(' - ')[1])
    res['type'] = res['stream'].apply(lambda x :  get_type(x))
    res['probability'] = res['stream'].apply(lambda x :  get_probability(x))
    for inf in info_from_hyperparameters:
        res[inf] = res['hyperparameters'].apply(lambda x : x[inf])
    #res['classifier'] = res['hyperparameters'].apply(lambda x : x['classifier'])

    #change order
    new_order = list(res.columns[:2])
    new_order.extend(res.columns[-(3+len(info_from_hyperparameters)):])
    new_order.extend(res.columns[2:-(3+len(info_from_hyperparameters))])
    res = res[new_order]

    return res


In [4]:
def save_formatted_logs(res,file):
    res.to_csv(os.path.join("logs_formatted",file),index = False)

In [5]:
def concatenate_res(res,file):
    new_res = pd.read_csv(os.path.join("logs_formatted",file))
    res = pd.concat([new_res,res])
    res.to_csv(os.path.join("logs_formatted",file),index = False)

In [6]:
logs_file = os.listdir('logs')[1]

In [108]:
os.listdir('logs')

['18_04_2023_22_21_30.log',
 '19_04_2023_20_35_10.log',
 '19_04_2023_21_00_24.log']

In [41]:
logs_file

'16_04_2023_16_32_33.log'

In [107]:
logs_file = os.listdir('logs')[0]
res = prepare_logs(logs_file,['threshold'])
res

,stream,hyperparameters,date,type,probability,threshold,B,frequency,B_1,B_2,...,m_390,m_391,m_392,m_393,m_394,m_395,m_396,m_397,m_398,m_399
0,LED_abrubt_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",18-Apr-23 22:47:10,lfs,0.1,0.9,50,100,0.000000,0.000000,...,,,,,,,,,,
1,initail_LED_Drift_abrubt,"{'grace_period': '200', 'max_depth': 'inf', 's...",18-Apr-23 22:47:44,init,None,0.9,50,100,0.000000,0.000000,...,,,,,,,,,,
2,LED_abrubt_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",18-Apr-23 22:49:47,lfs,0.1,0.8,50,100,0.000000,0.000000,...,,,,,,,,,,
3,initail_LED_Drift_abrubt,"{'grace_period': '200', 'max_depth': 'inf', 's...",18-Apr-23 22:51:02,init,None,0.8,50,100,0.000000,0.000000,...,,,,,,,,,,
4,LED_abrubt_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",18-Apr-23 22:52:26,lfs,0.1,0.7,50,100,0.000000,0.000000,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,LED_gradual_constant_delay_ssl_0.1_0_20000,"{'threshold': '0.8', 'train_period': '0', 'cla...",19-Apr-23 23:30:26,ssl,0.1,0.8,50,100,0.451389,0.449299,...,,,,,,,,,,
68,LED_gradual_constant_delay_ssl_0.5_0_20000,"{'threshold': '0.8', 'train_period': '0', 'cla...",20-Apr-23 01:07:03,ssl,0.5,0.8,50,100,0.343161,0.336316,...,,,,,,,,,,
69,LED_gradual_constant_delay_ssl_0.2_0_20000,"{'threshold': '0.7', 'train_period': '0', 'cla...",20-Apr-23 01:27:04,ssl,0.2,0.7,50,100,0.427657,0.426894,...,,,,,,,,,,
70,LED_gradual_constant_delay_ssl_0.1_0_20000,"{'threshold': '0.7', 'train_period': '0', 'cla...",20-Apr-23 01:37:12,ssl,0.1,0.7,50,100,0.409830,0.408027,...,,,,,,,,,,


In [103]:
res.iloc[48:,0]

In [ ]:
res.iloc[48:,:-200]

,stream,hyperparameters,date,type,probability,threshold,B,frequency,B_1,B_2,...,m_190,m_191,m_192,m_193,m_194,m_195,m_196,m_197,m_198,m_199
48,RBF_moderate_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:16:51,lfs,0.1,0.6,50,100,0.000000,0.000000,...,,,,,,,,,,
49,RBF_moderate_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:17:36,lfs,0.1,0.5,50,100,0.000000,0.000000,...,,,,,,,,,,
50,RBF_moderate_constant_delay_lfs_0.1_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:18:10,lfs,0.1,0.4,50,100,0.000000,0.000000,...,,,,,,,,,,
51,initail_RBF_moderate,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:18:15,init,None,0.6,50,100,0.000000,0.000000,...,0.65,0.76,0.81,0.87,0.77,0.84,0.73,0.77,0.82,0.72
52,RBF_moderate_constant_delay_lfs_0.2_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:18:42,lfs,0.2,0.6,50,100,0.000000,0.000000,...,,,,,,,,,,
53,RBF_moderate_constant_delay_lfs_0.2_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:19:24,lfs,0.2,0.5,50,100,0.000000,0.000000,...,,,,,,,,,,
54,RBF_moderate_constant_delay_lfs_0.2_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:20:03,lfs,0.2,0.4,50,100,0.000000,0.000000,...,,,,,,,,,,
55,initail_RBF_moderate,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:21:09,init,None,0.5,50,100,0.000000,0.000000,...,0.72,0.76,0.79,0.85,0.78,0.85,0.75,0.76,0.86,0.71
56,initail_RBF_moderate,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:24:25,init,None,0.4,50,100,0.000000,0.000000,...,0.6,0.7,0.72,0.67,0.73,0.75,0.74,0.7,0.79,0.71
57,RBF_moderate_constant_delay_lfs_0.5_0_20000,"{'grace_period': '200', 'max_depth': 'inf', 's...",15-Apr-23 19:25:34,lfs,0.5,0.6,50,100,0.000000,0.000000,...,,,,,,,,,,


In [85]:
concatenate_res(res.iloc[24:,:-200],"RBF_moderate\\NB.csv")

In [105]:
save_formatted_logs(res.iloc[48:,:-200],"LED_abrubt\\NB.csv")

In [ ]:
res.iloc[28]['hyperparameters']['classifier']

"<class 'river.naive_bayes.gaussian.GaussianNB'>"